# NLP Project - A.A. 2024/25

Authors:

- Gigante Davide (11018245)
- Puccia Niccolò (10829496)
- Sichili Giulio (11016179)
- Troiano Alessandro (10776474)

Link to the recording: AAAA

# WORKFLOW
Possiamo provare a classificare le subjects (che sono 3, più semplice), o i topics, che sono di più e tendono a sovrapporsi più spesso. Partiamo dalla prima.

PRIMA PARTE
- Carica i dati
- Manteniamo le colonne che ci servono per la classificazione
- Eseguiamo operazioni sui dati che servono per pulire il dataset qualunque modello useremo per la classificazione




# Multimodal Question Answering: ScienceQA

## Connect to Google Drive

In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')

folder = "/content/drive/My Drive/NLP project"
os.chdir(folder)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Install Dependencies and libraries

In [2]:
!pip install -U datasets
!pip install --upgrade gensim

In [3]:
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from scipy.stats import mode
import re
import string


import nltk
from nltk.corpus import stopwords, wordnet
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
import plotly.express as px

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [4]:
ds = load_dataset("derek-thomas/ScienceQA")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

(…)-00000-of-00001-1028f23e353fbe3e.parquet:   0%|          | 0.00/377M [00:00<?, ?B/s]

(…)-00000-of-00001-6c7328ff6c84284c.parquet:   0%|          | 0.00/126M [00:00<?, ?B/s]

(…)-00000-of-00001-f0e719df791966ff.parquet:   0%|          | 0.00/122M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12726 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4241 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4241 [00:00<?, ? examples/s]

In [5]:
ds

DatasetDict({
    train: Dataset({
        features: ['image', 'question', 'choices', 'answer', 'hint', 'task', 'grade', 'subject', 'topic', 'category', 'skill', 'lecture', 'solution'],
        num_rows: 12726
    })
    validation: Dataset({
        features: ['image', 'question', 'choices', 'answer', 'hint', 'task', 'grade', 'subject', 'topic', 'category', 'skill', 'lecture', 'solution'],
        num_rows: 4241
    })
    test: Dataset({
        features: ['image', 'question', 'choices', 'answer', 'hint', 'task', 'grade', 'subject', 'topic', 'category', 'skill', 'lecture', 'solution'],
        num_rows: 4241
    })
})

### Suppress Warnings

In [6]:
import warnings
warnings.filterwarnings("ignore")

# 1. Data Preprocessing

### Importing utilities

In [7]:
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True


### Cleaning utilities
Compiled objects live at module scope for maximum speed.



In [8]:
import re, string
from nltk.corpus import stopwords
from nltk import word_tokenize

# Try simple_preprocess from gensim, because we need the same preprocessing of the Word2Vec model

# Setup
_PUNCT_RE   = re.compile(f"[{re.escape(string.punctuation)}]")
_STOP_WORDS = set(stopwords.words("english"))

def clean_text(text: str) -> list[str]:
    """clean = _PUNCT_RE.sub("", text.lower())
    tokens = [t for t in word_tokenize(clean) if t.isalpha() and t not in _STOP_WORDS]"""
    tokens = simple_preprocess(text, deacc=True)
    return tokens  # no lemmatization

def process_batch(batch):
    batch["question"] = [clean_text(q) for q in batch["question"]]
    batch["choices"] = [[clean_text(c) for c in choice_list] for choice_list in batch["choices"]]
    return batch

### Take only the columns needed for classification

In [9]:
# Applica la funzione
columns_X = ["question", "choices"]
possible_y = ["subject", "topic"]
columns_needed = columns_X + possible_y
print(columns_needed)
ds_cleaned = ds.map(
    process_batch,
    batched=True,
    remove_columns=[col for col in ds["train"].features if col not in columns_needed]
)

['question', 'choices', 'subject', 'topic']


Map:   0%|          | 0/12726 [00:00<?, ? examples/s]

Map:   0%|          | 0/4241 [00:00<?, ? examples/s]

Map:   0%|          | 0/4241 [00:00<?, ? examples/s]

# LSTM

In [10]:
from copy import deepcopy

ds_LSTM = deepcopy(ds_cleaned)

## Word2Vec embedding if needed by the model

In [11]:
from gensim.models import KeyedVectors

embeddings = KeyedVectors.load("data/embeddings.kv")
#print embeddings size
print(embeddings.vectors.shape)

(13411, 100)


## Print some examples

In [12]:
for word in ds_LSTM["train"]["question"][0]:
    print("Word: ", word, "\n embedding: ", embeddings[word])

Word:  which 
 embedding:  [ 0.2504122   0.38475913  0.15779835  0.33116853  0.5123317  -0.38409096
 -0.37433088  0.34925556 -0.35435256  0.01887937 -0.2613026   0.02672859
 -0.13926007  0.5182378  -0.14344281 -0.0528793   0.11406546  0.33018926
 -0.65141404 -0.36740893 -0.37070826  0.6881496   0.19559784  0.03126173
 -0.79936206  0.5368398  -0.5053665  -0.40091926 -0.04215838 -0.1071024
  0.24793358  0.27941355 -0.35790893 -0.123183   -0.08374839  0.30822465
  1.069571   -0.6685113  -0.2357762  -0.69836897  0.279269   -0.870108
 -0.56254387  0.16138874 -0.3526567  -0.26099747 -0.38674343 -0.35277244
  1.0068446  -0.08785898  0.1206533   0.09530291 -0.33050478  0.23763195
  0.059082   -0.4151822  -0.78517205 -0.63884133 -0.55050975  0.22420038
 -0.09599239 -0.4873313  -0.21119452 -0.4088446  -0.14524634  0.9776742
  0.6152476   0.48393148  0.19863378 -0.16927026 -0.62063867  0.3421933
  0.06066138 -0.41816497  0.36009705 -0.3515809   0.08165442  0.33648247
 -0.69671726 -0.06772067  0.3

In [13]:
print(ds_LSTM)

DatasetDict({
    train: Dataset({
        features: ['question', 'choices', 'subject', 'topic'],
        num_rows: 12726
    })
    validation: Dataset({
        features: ['question', 'choices', 'subject', 'topic'],
        num_rows: 4241
    })
    test: Dataset({
        features: ['question', 'choices', 'subject', 'topic'],
        num_rows: 4241
    })
})


# Let's use Subject as a label

In [53]:
selected_label = "subject"
not_selected_label = "topic"
# drop not selected label
ds_LSTM = ds_LSTM.map(
    lambda x: x,
    batched=True,
    remove_columns=[not_selected_label] if not_selected_label in ds_LSTM["train"].features else []
)
# Separate features from label
X = ds_LSTM.map(
    lambda x: x,
    batched=True,
    remove_columns=[selected_label] if selected_label in ds_LSTM["train"].features else []
)
print(X)
y_train, y_validation, y_test = ds_cleaned["train"]["subject"], ds_cleaned["validation"]["subject"], ds_cleaned["test"]["subject"]


Map:   0%|          | 0/12726 [00:00<?, ? examples/s]

Map:   0%|          | 0/4241 [00:00<?, ? examples/s]

Map:   0%|          | 0/4241 [00:00<?, ? examples/s]

Map:   0%|          | 0/12726 [00:00<?, ? examples/s]

Map:   0%|          | 0/4241 [00:00<?, ? examples/s]

Map:   0%|          | 0/4241 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'choices'],
        num_rows: 12726
    })
    validation: Dataset({
        features: ['question', 'choices'],
        num_rows: 4241
    })
    test: Dataset({
        features: ['question', 'choices'],
        num_rows: 4241
    })
})


In [54]:
from sklearn.preprocessing import LabelBinarizer

# Crea e addestra l'encoder su tutte le etichette disponibili
encoder = LabelBinarizer()
encoder.fit(y_train + y_validation + y_test)  # tutte insieme per coerenza

# Trasforma le label in vettori one-hot
y_train = encoder.transform(y_train)
y_val   = encoder.transform(y_validation)
y_test  = encoder.transform(y_test)

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(12726, 3)
(4241, 3)
(4241, 3)


In [58]:
# B) domanda + choices in un’unica stringa (esempio)
def fuse(sample):
    # Convert the question and choices to strings before joining
    question_str = " ".join(sample["question"])
    choices_str = " ".join([" ".join(choice) for choice in sample["choices"]])
    return " ".join([question_str, choices_str])

train_x = [fuse(s) for s in X["train"]]   # list[str]
valid_x = [fuse(s) for s in X["validation"]]
test_x  = [fuse(s) for s in X["test"]]

## Install Tensorflow to use LSTM

In [59]:
# TensorFlow 2 and tf.keras
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.18.0


In [63]:
# creo dataset tensorflow
train_tf = tf.data.Dataset.from_tensor_slices((train_x, y_train))
valid_tf = tf.data.Dataset.from_tensor_slices((valid_x, y_val))
test_tf = tf.data.Dataset.from_tensor_slices((test_x, y_test))

In [64]:
train_dataset = train_tf.shuffle(buffer_size=10000).batch(batch_size=64).prefetch(tf.data.AUTOTUNE)
valid_dataset = valid_tf.batch(batch_size=64).prefetch(tf.data.AUTOTUNE)

In [65]:
for batch in train_dataset.take(1):
  print(batch)

(<tf.Tensor: shape=(64,), dtype=string, numpy=
array([b'why might raising cubs with other lionesses in pride increase an african lioness reproductive success complete the claim below that answers this question and is best supported by the passage raising cubs with other lionesses in pride increases the chances that the lioness cubs will be around other cubs the lioness cubs will survive attacks the lioness will feed the cubs of other lionesses',
       b'according to newton third law what other force must be happening the door is pulling on kelsey hand the door is pushing on kelsey hand',
       b'what do these two changes have in common newly poured concrete becoming hard copper statue turning green both are caused by heating both are chemical changes both are only physical changes both are caused by cooling',
       b'does this passage describe the weather or the climate climate weather',
       b'which is this organism scientific name frilled lizard chlamydosaurus kingii',
       b'

In [66]:
from tensorflow.keras.layers import TextVectorization
vectorizer = TextVectorization(max_tokens=5000)
train_text = train_dataset.map(lambda text, label: text)
vectorizer.adapt(train_text)

In [67]:
vocab = vectorizer.get_vocabulary()
vocab[:100]

['',
 '[UNK]',
 'the',
 'is',
 'of',
 'in',
 'which',
 'to',
 'what',
 'this',
 'that',
 'and',
 'on',
 'for',
 'are',
 'it',
 'with',
 'has',
 'can',
 'or',
 'force',
 'have',
 'following',
 'magnetic',
 'both',
 'sentence',
 'these',
 'was',
 'an',
 'does',
 'each',
 'his',
 'by',
 'pair',
 'same',
 'her',
 'more',
 'would',
 'do',
 'select',
 'at',
 'text',
 'will',
 'trait',
 'when',
 'capital',
 'as',
 'trade',
 'temperature',
 'you',
 'about',
 'new',
 'from',
 'water',
 'magnitude',
 'sample',
 'changes',
 'true',
 'word',
 'used',
 'than',
 'tense',
 'how',
 'information',
 'they',
 'he',
 'only',
 'not',
 'solution',
 'two',
 'magnets',
 'caused',
 'be',
 'she',
 'no',
 'common',
 'also',
 'acquired',
 'their',
 'highlighted',
 'tomatoes',
 'inherited',
 'find',
 'states',
 'one',
 'complete',
 'between',
 'higher',
 'particles',
 'animal',
 'long',
 'statement',
 'page',
 'words',
 'made',
 'supports',
 'weather',
 'other',
 'dictionary',
 'guide']

In [68]:
from tensorflow.keras import Sequential
model = tf.keras.Sequential([
    tf.keras.Input(shape=(1,), dtype=tf.string),
    vectorizer,
    tf.keras.layers.Embedding(input_dim=len(vectorizer.get_vocabulary()), output_dim=64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'], optimizer=tf.keras.optimizers.Adam(1e-4))


In [69]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (None, None)           │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 64)       │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 394,369 (1.50 MB)

 Trainable params: 394,369 (1.50 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(train_dataset, epochs=10, validation_data=valid_dataset, validation_steps=20)

In [ ]:
valid_loss, valid_acc = model.evaluate(valid_dataset)

print('Validation Loss: {}'.format(valid_loss))
print('Validation Accuracy: ',valid_acc)

In [ ]:
pred_y = [0 if (pred < 0) else 1 for pred in model.predict(tf.convert_to_tensor(valid_x))]

from sklearn.metrics import accuracy_score
print('accuracy: '+ str(accuracy_score(pred_y, valid_y)))

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cmd = ConfusionMatrixDisplay(confusion_matrix(valid_y, pred_y,normalize='true'), display_labels=['negative', 'positive'])
cmd.plot()

In [47]:
"""def concatenate_columns(batch):
    # batch["question"] e batch["choices"] sono liste batched
    flat = [
        q + [w for choice in c for w in choice]
        for q, c in zip(batch["question"], batch["choices"])
    ]
    return {"flat": flat}  # restituisco un dict

# Applico in batched=True
X_flat = X.map(
    concatenate_columns,
    batched=True,
    remove_columns=["choices", "question"]  # o togli richieste se vuoi tenere anche question/choices
)

# Ora
X_flat["train"].column_names"""

Map:   0%|          | 0/12726 [00:00<?, ? examples/s]

Map:   0%|          | 0/4241 [00:00<?, ? examples/s]

Map:   0%|          | 0/4241 [00:00<?, ? examples/s]

['flat']

In [20]:
"""VOCABULARY_SIZE, EMBEDDING_SIZE = embeddings.vectors.shape

def embed(word):
    return np.array(embeddings[word]) if word in embeddings else np.zeros(EMBEDDING_SIZE, dtype=np.float32)

def apply_embedding_batched(batch):
    batch["question"] = [
        [embed(word) for word in question] for question in batch["question"]
    ]
    batch["choices"] = [
        [embed(word) for choice in choices for word in choice]
        for choices in batch["choices"]
    ]
    return batch



X_embedded = X.map(
    lambda x: apply_embedding_batched(x),
    batched=True
)



print(X_embedded)"""

Map:   0%|          | 0/12726 [00:00<?, ? examples/s]

Map:   0%|          | 0/4241 [00:00<?, ? examples/s]

Map:   0%|          | 0/4241 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'choices'],
        num_rows: 12726
    })
    validation: Dataset({
        features: ['question', 'choices'],
        num_rows: 4241
    })
    test: Dataset({
        features: ['question', 'choices'],
        num_rows: 4241
    })
})
